<!-- dom:TITLE: Finite difference, linear algebra, and tridiagonal matrices  -->
# Finite difference, linear algebra, and tridiagonal matrices 
<!-- dom:AUTHOR: Prepared as part of MOD510 Computational Engineering and Modeling -->
<!-- Author: -->  
**Prepared as part of MOD510 Computational Engineering and Modeling**

Date: **Feb 8, 2022**

# Solving the heat equation

Learning objectives:
1. Understand the origin of a conservation equation

2. Formulate a finite difference problem as a matrix inversion problem

3. Quantify the numerical error, investigate the scaling, and analyze the error using Taylors formula 

4. Use a sparse solver to speed up the calculation

<!-- --- begin exercise --- -->

## Exercise 1: Conservation Equation or the Continuity Equation

<!-- dom:FIGURE: [fig-lin/heat.png, width=700 frac=.9] Conservation of energy and the continuity equation. <div id="fig:nlin:heat"></div> -->
<!-- begin figure -->
<div id="fig:nlin:heat"></div>

<p>Conservation of energy and the continuity equation.</p>
<img src="fig-lin/heat.png" width=700>

<!-- end figure -->


In [figure](#fig:nlin:heat), the continuity equation is derived for
heat flow.
### Heat equation for solids

As derived in the beginning of this chapter the heat equation for a solid is

<!-- Equation labels as ordinary links -->
<div id="eq:nlin:heateq"></div>

$$
\begin{equation}
\frac{d^2T}{dx^2}+\frac{\dot{\sigma}}{k}=\frac{\rho c_p}{k}\frac{dT}{dt},
\label{eq:nlin:heateq} \tag{1}
\end{equation}
$$

where $\dot{\sigma}$ is the rate of heat generation in the solid. This
equation can be used as a starting point for many interesting
models. In this exercise we will investigate the *steady state*
solution, *steady state* is just a fancy way of expressing that we
want the solution that *does not change with time*. This is achieved
by ignoring the derivative with respect to time in equation
([1](#eq:nlin:heateq)). We want to study a system with size $L$, and is
it good practice to introduce a dimensionless variable: $y=x/L$. 


**Part 1.**

Show that equation ([1](#eq:nlin:heateq)) now takes the following form:

<!-- Equation labels as ordinary links -->
<div id="eq:nlin:heat2"></div>

$$
\begin{equation}
\frac{d^2T }{dy^2}+\frac{\dot{\sigma}L^2}{k}=0
\label{eq:nlin:heat2} \tag{2}
\end{equation}
$$

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 2: Curing of Concrete and Matrix Formulation

Curing of concrete is one particular example that we can investigate
with equation ([2](#eq:nlin:heat2)). When concrete is curing, there are
a lot of chemical reactions happening, these reactions generate
heat. This is a known issue, and if the temperature rises too much 
compared to the surroundings, the concrete may fracture.  In the
following we will, for simplicity, assume that the rate of heat
generated during curing is constant, $\dot{\sigma}=$100 W/m$^3$. The
left end (at $x=0$) is insulated, meaning that there is no flow of
heat over that boundary, hence $dT/dx=0$ at $x=0$. On the right hand
side the temperature is kept constant, $x(L)=y(1)=T_1$, assumed to be
equal to the ambient temperature of $T_1=25^\circ$C.  The concrete
thermal conductivity is assumed to be $k=1.65$ W/m$^\circ$C.



**Part 1.**

Show that the solution to equation ([2](#eq:nlin:heat2)) in this case is:

<!-- Equation labels as ordinary links -->
<div id="eq:nlin:heatsol"></div>

$$
\begin{equation}
T(y)=\frac{\dot{\sigma}L^2}{2k}(1-y^2)+T_1.
\label{eq:nlin:heatsol} \tag{3}
\end{equation}
$$

**Part 2.**
In order to solve equation ([2](#eq:nlin:heat2)) numerically, we need to discretize
it. Show that equation ([2](#eq:nlin:heat2)) now takes the following form:

<!-- Equation labels as ordinary links -->
<div id="eq:nlin:heat3"></div>

$$
\begin{equation}
T_{i+1}+T_{i-1}-2T_i=-h^2\beta,
\label{eq:nlin:heat3} \tag{4}
\end{equation}
$$

where $\beta=\dot{\sigma}L^2/k$.
<!-- dom:FIGURE: [fig-lin/heat_grid.png, width=200 frac=.5] Finite difference grid for $N=4$. <div id="fig:nlin:hgrid"></div>  -->
<!-- begin figure -->
<div id="fig:nlin:hgrid"></div>

<p>Finite difference grid for $N=4$.</p>
<img src="fig-lin/heat_grid.png" width=200>

<!-- end figure -->


In [figure](#fig:nlin:hgrid), the finite difference grid is shown for
$N=4$.

**Part 3.**

Show that equation ([4](#eq:nlin:heat3)) including the boundary conditions for $N=4$ can be written as the following matrix equation

<!-- Equation labels as ordinary links -->
<div id="eq:lin:heats"></div>

$$
\begin{equation}
\left(
\begin{array}{cccc}
-\gamma&\gamma&0&0\\ 
1&-2&1&0\\ 
0&1&-2&1\\ 
0&0&1&-2\\ 
\end{array}
\right)
\left(
\begin{array}{c}
T_0\\ 
T_1\\ 
T_2\\ 
T_3\\ 
\end{array}
\right)
=
\left(
\begin{array}{c}
-h^2\beta\\ 
-h^2\beta\\ 
-h^2\beta\\ 
-h^2\beta-25
\end{array}
\right).
\end{equation}
\label{eq:lin:heats} \tag{5}
$$

where $\gamma=2$ for the central difference scheme and 1 for the forward difference scheme.


**Part 4.**
* Solve the set of equations in equation ([5](#eq:lin:heats)) using [`numpy.linalg.solve`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html).

* Write the code so that you can easily switch between the central difference scheme and forward difference

* Evaluate the numerical error as you change $h$, how does it scale? Is it what you expect?

In [1]:
%matplotlib inline

import numpy as np
import scipy as sc
import scipy.sparse.linalg
from numpy.linalg import solve
import matplotlib.pyplot as plt

In [2]:

central_difference=False
# set simulation parameters
h=0.25
L=1.0
n = int(round(L/h))
Tb=25 #rhs
sigma=100
k=1.65 
beta = sigma*L**2/k

y = np.arange(n+1)*h

def analytical(x):
    return beta*(1-x*x)/2+Tb
def tri_diag(a, b, c, k1=-1, k2=0, k3=1):
    """ a,b,c diagonal terms
        default k-values for 4x4 matrix:
        | b0 c0 0  0 |
        | a0 b1 c1 0 |
        | 0  a1 b2 c2|
        | 0  0  a2 b3|
    """
    return np.diag(a, k1) + np.diag(b, k2) + np.diag(c, k3)
# defina a, b and c vector
a=np.ones(n-1)
b=..
c=..

if central_difference:
    c[0]= ...
else:
    b[0]=...

A=tri_diag(a,b,c)
print(A) # view matrix - compare with N=4 to make sure no bugs
# define rhs vector
d=...
#rhs boundary condition
d[-1]=...

Tn=np.linalg.solve(A,d)
print(Tn)

The correct solution for $L=1$ m, and $h=1/4$, is: $[T_0,T_1.T_2,T_3]$=[55.3030303 , 53.40909091, 47.72727273, 38.25757576] (central difference) and $[T_0,T_1.T_2,T_3]$=[62.87878788, 59.09090909, 51.51515152, 40.15151515] (forward difference)

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 3: Solve the full heat equation

**Part 1.**
Replace the time derivative in equation ([1](#eq:nlin:heateq)) with

<!-- Equation labels as ordinary links -->
<div id="eq:lin:dt"></div>

$$
\begin{equation}
\frac{dT}{dt}\simeq\frac{T(t+\Delta t)-T(t)}{\Delta t}=\frac{T^{n+1}-T^n}{\Delta t}, 
\label{eq:lin:dt} \tag{6}
\end{equation}
$$

and show that by using an *implicit formulation* (i.e. that the second derivative with respect to $x$ is to be evaluated at $T(t+\Delta t)\equiv T^{n+1}$) that equation ([1](#eq:nlin:heateq)) can be written

<!-- Equation labels as ordinary links -->
<div id="eq:lin:imp"></div>

$$
\begin{equation}
T_{i+1}^{n+1}+T_{i-1}^{n+1}-(2+\frac{\alpha h^2}{\Delta t})T_i^{n+1}=-h^2\beta-\frac{\alpha h^2 }{\Delta t}T_i^n,
\label{eq:lin:imp} \tag{7} 
\end{equation}
$$

where $\alpha\equiv\rho c_p/k$.

**Part 2.**

Use the central difference formulation for the boundary condition and show that for four nodes we can formulate equation ([7](#eq:lin:imp)) as the following matrix equation

<!-- Equation labels as ordinary links -->
<div id="_auto1"></div>

$$
\begin{equation}
\left(
\begin{array}{cccc}
-(2+\frac{\alpha h^2}{\Delta t})&2&0&0\\ 
1&-(2+\frac{\alpha h^2}{\Delta t})&1&0\\ 
0&1&-(2+\frac{\alpha h^2}{\Delta t})&1\\ 
0&0&1&-(2+\frac{\alpha h^2}{\Delta t})\\ 
\end{array}
\right)
\left(
\begin{array}{c}
T_0^{n+1}\\ 
T_1^{n+1}\\ 
T_2^{n+1}\\ 
T_3^{n+1}\\ 
\end{array}
\right){\nonumber}
\label{_auto1} \tag{8}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="eq:lin:heatfull"></div>

$$
\begin{equation}  
=
\left(
\begin{array}{c}
-h^2\beta\\ 
-h^2\beta\\ 
-h^2\beta\\ 
-h^2\beta-25
\end{array}
\right)
-\frac{\alpha h^2 }{\Delta t}
\left(
\begin{array}{c}
T_0^n\\ 
T_1^n\\ 
T_2^n\\ 
T_3^n\\ 
\end{array}
\right)
\end{equation}
\label{eq:lin:heatfull} \tag{9}
$$

**Part 3.**
Assume that the initial temperature in the concrete is $25^\circ$C, $\rho$=2400 kg/m$^3$, a specific heat capacity $c_p=$ 1000 W/kg K, and a time step of $\Delta t=86400$ s (1 day). Solve equation ([9](#eq:lin:heatfull)), plot the result each day and compare the result after 50 days with the steady state solution in equation ([3](#eq:nlin:heatsol)).

<!-- --- end exercise --- -->




<!-- --- begin exercise --- -->

## Exercise 4: Using sparse matrices in python

In this part we are going to create a sparse matrix in python and use `scipy.sparse.linalg.spsolve` to solve it. The matrix is created using `scipy.sparse.spdiags`.


**Part 1.**
Extend the code you developed in the last exercises to also be able to use sparse matrices, by e.g. a logical switch. Sparse matrices may be defined as follows

In [3]:
import scipy.sparse.linalg

#right hand side
# rhs vector
d=np.repeat(-h*h*beta,n)
#rhs - constant temperature
Tb=25
d[-1]=d[-1]-Tb
#Set up sparse matrix
diagonals=np.zeros((3,n))
diagonals[0,:]= 1
diagonals[1,:]= -2  
diagonals[2,:]= 1
#No flux boundary condition
diagonals[2,1]= 2
A_sparse = sc.sparse.spdiags(diagonals, [-1,0,1], n, n,format='csc')
# to view matrix - do this and check that it is correct!
print(A_sparse.todense())
# solve matrix
Tb = sc.sparse.linalg.spsolve(A_sparse,d)

# if you like you can use timeit to check the efficiency
# %timeit sc.sparse.linalg.spsolve( ... )

* Compare the sparse solver with the standard Numpy solver using
  `%timeit`, how large must the linear system be before an improvement
  in speed is seen?

<!-- --- end exercise --- -->